In [25]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import re
import pandas as pd

In [27]:
ementa = list()
processo = list()
tipo = list()
agencia_envolvida = list()
relator = list()
turma = list()
data_julgamento = list()
data_publicacao = list()
agencia_envolvida = list()

agencias = ['ANATEL', 'ANVISA', 'ANEEL', 'ANAC','ANTT', 'ANS', 'ANTAQ', 'ANP']
for agencia in agencias:
    path = 'D:/papers/direito/chromedriver_win32'
    final_path = 'D:/papers/direito/scrapping'

    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1240,1080") # define o tamanho da janela a ser aberta
    driver = webdriver.Chrome(executable_path = os.path.join(path,'chromedriver.exe'), options = chrome_options) # o path do chromedriver
    url = 'https://scon.stj.jus.br/SCON/pesquisar.jsp'
    driver.get(url)
    
    pesquisa = driver.find_elements(by=By.XPATH, value='//*[@id="pesquisaLivre"]')
    #time.sleep(5)
    pesquisa[0].send_keys(f'"Recurso Especial" {agencia} Nao embargo nao agravo') # buscando os acórdãos desejados

    pesquisa_botao = driver.find_element(by=By.XPATH, value='//*[@id="frmConsulta"]/div[3]/div[2]/div[2]/div/div/button') # salvando o botão de clique de busca
    #time.sleep(3)
    pesquisa_botao.click() # clicando na busca

    for pagina in range(20):
        
        html_source = driver.page_source
        soup = BeautifulSoup(html_source,"html.parser") # salvando a página encontrada pela busca

        aumentar_acordaos = Select (driver.find_element(by=By.XPATH, value='//*[@id="qtdDocsPagina"]')) # expandindo o número de acórdãos por página
        aumentar_acordaos.select_by_value('50')
        
        simplificaco_botao = driver.find_element(by=By.XPATH, value='//*[@id="tp_vis_lista_resumida"]/span') # esse botão simplificará a página, facilitando a busca pelo texto da ementa
        #time.sleep(3)
        simplificaco_botao.click()
    
        html_source = driver.page_source
        soup = BeautifulSoup(html_source,"html.parser") # salvando a página simplificada

        for i in range(0,len(soup.find_all('div',{'class':'row itemlistadocumentos p-2'}))): # obtendo a ementa
            ementa.append(soup.find_all('div', {'class':'clsEmentaCompleta'})[i].get_text().replace('\n','').replace('\t',''))

        completo_botao = driver.find_element(by=By.XPATH, value='//*[@id="tp_vis_documento_completo"]/span') # esse botão volta a página à sua forma inicial, facilitando a busca por outros elementos
        #time.sleep(3)
        completo_botao.click()

        html_source = driver.page_source
        soup = BeautifulSoup(html_source,"html.parser") # salvando a página completa

        for i in range(0, len(soup.find_all('div',{'class':'documento'}))): 
            agencia_envolvida.append(agencia) # salvando a agência que está sendo buscada
            
        for i in range(0,len(soup.find_all('div',{'class':'col-md-6 col-sm-12'})),2):
            tipo.append(re.search('(?<=Processo)\D{2,4}(?=\d{0,})', re.sub('\s+', '', soup.find_all('div',{'class':'col-md-6 col-sm-12'})[0].get_text()))[0]) # tipo do processo
            processo.append(re.search('\d{3,}\s\/\s\w\w', soup.find_all('div',{'class':'col-md-6 col-sm-12'})[i].get_text())[0]) # salvando o elemento com detalhes de identificação do processo

        for i in range(0,len(soup.find_all('div',{'class':'col-md-3 col-sm-12'})),4): # esse elemento é uma lista de quatro elementos semelhantes, então o contador pulará de quatro em quatro
            relator.append(re.subn('Relator[a]?', '', re.subn('\\n','',soup.find_all('div',{'class':'col-md-3 col-sm-12'})[0].get_text())[0])[0]) # salvando o relator
            turma.append(re.search('(?<=\\n\\nÓrgão Julgador\\n).*(?=\\n\\n)',soup.find_all('div',{'class':'col-md-3 col-sm-12'})[i + 1].get_text())[0]) # salvando a turma
            data_julgamento.append(re.search('\d\d\/\d\d\/\d\d\d\d',soup.find_all('div',{'class':'col-md-3 col-sm-12'})[i + 2].get_text())[0]) # salvando a data
            data_publicacao.append(re.search('\d\d\/\d\d\/\d\d\d\d',soup.find_all('div',{'class':'col-md-3 col-sm-12'})[i + 3].get_text())[0]) # salvando a data
        
        try:
            if pagina == 0:
                #time.sleep(3)
                driver.find_elements(by=By.XPATH, value='//*[@id="navegacao"]/div[2]/a[1]/span')[0].click() # clique para a próxima página quando se está na primeira página
            else:
                #time.sleep(3)
                driver.find_elements(by=By.XPATH, value='//*[@id="navegacao"]/div[2]/a[3]/span')[0].click() # clique para a próxima página quando não se está na primeira página
        except:
            break

    driver.quit()

dados1 = {'processo': processo,
    'tipo_processo': tipo,
    'ministro_relator': relator,
    'orgao_julgador': turma,
    'agencia': agencia_envolvida,
    'data_julgamento': data_julgamento,
    'data_publicacao': data_publicacao,
    'ementa': ementa
} # salvando todas as listas obtidas num dicionário

df1 = pd.DataFrame.from_dict(dados1).drop_duplicates()
df1 = df1[df1.tipo_processo == 'REsp'] # transformando o dicionário em um dataframe
df1.to_excel('D:/papers/direito/scrapping/base.xlsx',index=False)
df1

C:\Users\alves\AppData\Local\Temp/ipykernel_12356/613207730.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = os.path.join(path,'chromedriver.exe'), options = chrome_options) # o path do chromedriver


,processo,tipo_processo,ministro_relator,orgao_julgador,agencia,data_julgamento,data_publicacao,ementa
0,1874643 / RJ,REsp,Ministra NANCY ANDRIGHI (1118) para AcórdãoMin...,T3 - TERCEIRA TURMA,ANATEL,03/05/2022,20/05/2022,RECURSO ESPECIAL. 1. AÇÃO COLETIVA DE CONSUMO ...
1,1473550 / RJ,REsp,Ministra NANCY ANDRIGHI (1118) para AcórdãoMin...,T1 - PRIMEIRA TURMA,ANATEL,14/12/2021,04/02/2022,PROCESSUAL CIVIL. TRIBUTÁRIO. RECURSO ESPECIAL...
2,1474142 / RJ,REsp,Ministra NANCY ANDRIGHI (1118) para AcórdãoMin...,T1 - PRIMEIRA TURMA,ANATEL,14/12/2021,04/02/2022,PROCESSUAL CIVIL. TRIBUTÁRIO. RECURSO ESPECIAL...
3,1823010 / RS,REsp,Ministra NANCY ANDRIGHI (1118) para AcórdãoMin...,T2 - SEGUNDA TURMA,ANATEL,23/11/2021,25/11/2021,PROCESSUAL CIVIL. CITAÇÃO PESSOAL. PROCURADOR ...
4,1599065 / DF,REsp,Ministra NANCY ANDRIGHI (1118) para AcórdãoMin...,T1 - PRIMEIRA TURMA,ANATEL,09/11/2021,02/12/2021,PROCESSUAL CIVIL. TRIBUTÁRIO. RECURSO ESPECIAL...
...,...,...,...,...,...,...,...,...
533,714110 / RJ,REsp,Ministra REGINA HELENA COSTA (1157),T2 - SEGUNDA TURMA,ANP,13/09/2005,03/10/2005,"ADMINISTRATIVO - PORTARIA 202/99, DA ANP - COM..."
534,676172 / RJ,REsp,Ministra REGINA HELENA COSTA (1157),T1 - PRIMEIRA TURMA,ANP,02/06/2005,27/06/2005,ADMINISTRATIVO. RECURSO ESPECIAL. AQUISIÇÃO DE...
535,669740 / RJ,REsp,Ministra REGINA HELENA COSTA (1157),T1 - PRIMEIRA TURMA,ANP,17/02/2005,04/04/2005,RECURSO ESPECIAL. PROCESSUAL CIVIL. LITISPENDÊ...
536,6862 / RJ,MC,Ministro JOSÉ DELGADO (1105),T1 - PRIMEIRA TURMA,ANP,18/12/2003,31/05/2004,MEDIDA CAUTELAR. LICITAÇÃO. CONCESSÃO. EXPLORA...
